In [0]:
"""Link para obter o dataset: https://github.com/mcmaximiano/Homework5"""

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split as split
import tensorflow as tf

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import os

import keras as krs
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

!unzip fruits-processed.zip




In [0]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

In [0]:
width = 100
height = 100
channels = 3

import matplotlib.image as mpimg



test_image = mpimg.imread(os.path.join("fruits-processed","fruits-360", "Test","Orange","30_100.jpg"))[:, :, :channels]
""" Just a test
plt.imshow(test_image)
plt.axis("off")
plt.show()
"""

test_image = 2 * test_image - 1


In [0]:
one_time_flag = 0

In [0]:
import sys
import tarfile
from six.moves import urllib

"""
Get imagenet_class_names from https://github.com/ageron/handson-ml/blob/master/datasets/inception/imagenet_class_names.txt
"""

TF_MODELS_URL = "http://download.tensorflow.org/models"
INCEPTION_V3_URL = TF_MODELS_URL + "/inception_v3_2016_08_28.tar.gz"
INCEPTION_PATH = os.path.join("datasets", "inception")
INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")


def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()

def fetch_pretrained_inception_v3(url=INCEPTION_V3_URL, path=INCEPTION_PATH):
    if os.path.exists(INCEPTION_V3_CHECKPOINT_PATH):
        return
    os.makedirs(path, exist_ok=True)
    tgz_path = os.path.join(path, "inception_v3.tgz")
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
    inception_tgz = tarfile.open(tgz_path)
    inception_tgz.extractall(path=path)
    inception_tgz.close()
    os.remove(tgz_path)

In [16]:
fetch_pretrained_inception_v3()

Downloading: 100%

In [0]:
import re

CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)

""" At this point get the file 'inagenet_class_names.txt from:
https://goo.gl/brXRtZ and upload it into the inception folder
'"""

def load_class_names():
    with open(os.path.join("datasets", "inception", "imagenet_class_names.txt"), "rb") as f:
        content = f.read().decode("utf-8")
        return CLASS_NAME_REGEX.findall(content)

In [0]:
class_names = ["background"] + load_class_names()

In [19]:
class_names[:5]

['background',
 'tench, Tinca tinca\r',
 'goldfish, Carassius auratus\r',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias\r',
 'tiger shark, Galeocerdo cuvieri\r']

In [0]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

reset_graph()

X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="X")

with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(
       X, num_classes=1001, is_training=False)
predictions = end_points["Predictions"]
saver = tf.train.Saver()

In [100]:
with tf.Session() as sess:
    saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)

INFO:tensorflow:Restoring parameters from datasets/inception/inception_v3.ckpt


In [122]:

test_image.resize((1, 299, 299, channels))

with tf.Session() as sess:
    saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    predictions_val = predictions.eval(feed_dict={X: test_image})

(1, 299, 299, 3)
INFO:tensorflow:Restoring parameters from datasets/inception/inception_v3.ckpt


In [123]:
most_likely_class_index = np.argmax(predictions_val[0])
most_likely_class_index

917

In [124]:
class_names[most_likely_class_index]

'web site, website, internet site, site\r'

In [125]:
top_5 = np.argpartition(predictions_val[0], -5)[-5:]
top_5 = reversed(top_5[np.argsort(predictions_val[0][top_5])])
for i in top_5:
    print("{0}: {1:.2f}%".format(class_names[i], 100 * predictions_val[0][i]))

: 100.00%
: 0.00%
: 0.00%
: 0.00%
: 0.00%


In [126]:
import sys
import tarfile

"""
We are only running this on the Training folder because everything is just too much to process
in useful time. But in code below we separate the images in this folder into a training set 
and a test set :)
"""
root_path = os.path.join("fruits-processed", "fruits-360", "Training")

fruit_classes = sorted([dirname for dirname in os.listdir(root_path) 
                         if os.path.isdir(os.path.join(root_path, dirname))])
fruit_classes

fruits-processed/fruits-360/Training


['Apple', 'Lemon', 'Orange', 'Pear']

In [0]:
from collections import defaultdict

image_paths = defaultdict(list)

for fruit_class in fruit_classes:
    image_dir = os.path.join(root_path, fruit_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[fruit_class].append(os.path.join(image_dir, filepath))

In [0]:
for paths in image_paths.values():
    paths.sort()    

In [0]:
import matplotlib.image as mpimg

n_examples_per_class = 2

for fruit_class in fruit_classes:
    print("Class:", fruit_class)
    plt.figure(figsize=(10,5))
    for index, example_image_path in enumerate(image_paths[fruit_class][:n_examples_per_class]):
        example_image = mpimg.imread(example_image_path)[:, :, :channels]
        plt.subplot(100 + n_examples_per_class * 10 + index + 1)
        plt.title("{}x{}".format(example_image.shape[1], example_image.shape[0]))
        plt.imshow(example_image)
        plt.axis("off")
    plt.show()

In [0]:
def resizeImg(image, target_width = 299, target_height = 299):
  new_img = image.copy()
  new_img.resize((299, 299,3))
  return new_img.astype(np.float32) / 255

In [0]:

plt.imshow(example_image)
plt.title("{}x{}".format(example_image.shape[1], example_image.shape[0]))
plt.axis("off")
plt.show()

In [0]:
prepared_image = resizeImg(example_image)

plt.imshow(prepared_image)
plt.title("{}x{}".format(prepared_image.shape[1], prepared_image.shape[0]))
plt.axis("off")
plt.show()

In [0]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

reset_graph()

X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()

In [190]:
end_points["PreLogits"]

<tf.Tensor 'InceptionV3/Logits/Dropout_1b/cond/Merge:0' shape=(?, 1, 1, 2048) dtype=float32>

In [0]:
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

In [0]:
n_outputs = len(fruit_classes)

with tf.name_scope("new_output_layer"):
    fruit_logits = tf.layers.dense(prelogits, n_outputs, name="fruit_logits")
    Y_proba = tf.nn.softmax(fruit_logits, name="Y_proba")

In [0]:
y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=fruit_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    fruit_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="fruit_logits")
    training_op = optimizer.minimize(loss, var_list=fruit_vars)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(fruit_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 

In [194]:
[v.name for v in fruit_vars]

['fruit_logits/kernel:0', 'fruit_logits/bias:0']

In [195]:
fruit_class_ids = {fruit_class: index for index, fruit_class in enumerate(fruit_classes)}
fruit_class_ids

{'Apple': 0, 'Lemon': 1, 'Orange': 2, 'Pear': 3}

In [0]:
fruit_paths_and_classes = []
for fruit_class, paths in image_paths.items():
    for path in paths:
        fruit_paths_and_classes.append((path, fruit_class_ids[fruit_class]))

In [0]:
test_ratio = 0.2
train_size = int(len(fruit_paths_and_classes) * (1 - test_ratio))

np.random.shuffle(fruit_paths_and_classes)

fruit_paths_and_classes_train = fruit_paths_and_classes[:train_size]
fruit_paths_and_classes_test = fruit_paths_and_classes[train_size:]

In [198]:
fruit_paths_and_classes_train[:3]

[('fruits-processed/fruits-360/Training/Pear/190_100.jpg', 3),
 ('fruits-processed/fruits-360/Training/Apple/r_247_100 (3).jpg', 0),
 ('fruits-processed/fruits-360/Training/Apple/117_100 (3).jpg', 0)]

In [0]:
from random import sample

def prepare_batch(fruit_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(fruit_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [resizeImg(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch

In [200]:
X_batch, y_batch = prepare_batch(fruit_paths_and_classes_train, batch_size=4)

(100, 100, 3)
(299, 299, 3)


In [202]:
X_test, y_test = prepare_batch(fruit_paths_and_classes_test, batch_size=len(fruit_paths_and_classes_test))

(100, 100, 3)
(299, 299, 3)


In [0]:
n_epochs = 10
batch_size = 40
n_iterations_per_epoch = len(fruit_paths_and_classes_train) // batch_size

with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)

    for epoch in range(n_epochs):
        print("Epoch", epoch, end="")
        for iteration in range(n_iterations_per_epoch):
            print(".", end="")
            X_batch, y_batch = prepare_batch(fruit_paths_and_classes_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})

        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print("  Train accuracy:", acc_train)

        save_path = saver.save(sess, "./my_fruit_model")

INFO:tensorflow:Restoring parameters from datasets/inception/inception_v3.ckpt
Epoch 0...............................................................................................................